In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Add this BEFORE any imports
import pandas as pd
from pathlib import Path
from database.neo4j import Neo4jHandler
from scripts.pdf_processor import PDFProcessor
from scripts.entities_extractor import EntityExtractor
from scripts.graph_loader import graphQLoader
from rag.models.langchainRag import RAGRetriever
from langchain_openai import ChatOpenAI
import json

In [2]:
#Init components
pdf_url = "https://www.pollitecon.com/html/ebooks/Workbook-4-The-Second-World-War.pdf"
client = Neo4jHandler()
pdf_processor = PDFProcessor()
extractor = EntityExtractor()
loader = graphQLoader(client)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

### GraphRAG

In [3]:
##THIS PART IS COMMENTED OUT TO AVOID RE-RUNNING THE PDF PROCESSING AND ENTITY EXTRACTION STEPS WHICH TAKES 29 MINUTES##

# # Load and process PDF
# text = pdf_processor.get_text_from_url(pdf_url)
# # Create chunks with IDs
# chunks = pdf_processor.chunk_text(text)
# print(f"Extracted {len(chunks)} chunks from the PDF.")
# #Extraxt entities and relations from chunks
# results = []
# entities = []
# relations = []
# for chunk in chunks:
#     print(f"Processing entities from chunk ID: {chunk['id']}")
#     result = extractor.extract_from_chunk(chunk)
#     results.append(result)

# for result in results:
#     entities.extend(result['entities'])
#     relations.extend(result['relations'])

# # Deduplicate entities
# dedup_entities = extractor.deduplicate_entities(entities)
# print(f"Extracted {len(dedup_entities)} unique entities and {len(relations)} relations.")

# # Save results to data directory
# extractor.save_results(chunks, dedup_entities, relations)

In [5]:
# Load entities and relations into Neo4j
loader.load_entities_to_neo4j()

Loading 1008 entities and 884 relations to Neo4j...
✅ Data loaded successfully!


In [3]:
qa_chain = RAGRetriever(client=client, llm=llm)

### Multimodal RAG

In [8]:
%%capture
from scripts.pdf_multimodal_processor import PDFMultimodalProcessor
from database.faiss import faissHandler
from rag.models.multimodalRag import multimodalRag

In [9]:
#Init components
processor = PDFMultimodalProcessor()
faiss = faissHandler()

In [10]:
#Process PDF and create vector store
docs, embeddingss, image_data_store = processor.process_pdf(url=pdf_url)
vector_store = faiss.create_faiss_index(docs, embeddingss)
rag = multimodalRag(image_data_store=image_data_store)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


### Comparisson between text-only RAG and multimodal RAG

In [4]:
# 10 Strategic Queries for RAG System Comparison
# WWII Document Analysis: Traditional RAG vs Graph RAG vs Multimodal RAG

queries = []

### **1. Simple Factual Query**
### *Favors: Traditional RAG with good compression*
queries.append("What did Metaxas say when Italy attacked Greece in October 1940?")

### **2. Leader Identification**
### *Favors: Traditional RAG with good retrieval*
queries.append("Who were the main resistance leaders in Albania during WWII?")

### **3. Basic Source Attribution**
### *Favors: Traditional RAG with reranking*
queries.append("What does Pavelić's photographer say about him?")

### **4. Direct Person-Organization Relationship**
### *Perfect for Graph RAG - 1-hop query*
queries.append("Who led the Yugoslav Partisans?")

### **5. Simple Entity Property Query**
### *Perfect for Graph RAG - direct node lookup*
queries.append("What was Tito's role in Yugoslavia?")

### **6. Basic Opposition Relationship**
### *Perfect for Graph RAG - direct relationship*
queries.append("Who opposed the German forces in Greece?")

### **7. Visual Reference Query**
### *Favors: Multimodal RAG*
queries.append("What do the photographs and posters in the document show about the war?")

### **8. Simple Numerical Data**
### *Favors: Multimodal RAG + Traditional RAG*
queries.append("What casualty numbers are mentioned in the document?")

### **9. Document Structure Query**
### *Favors: Multimodal RAG*
queries.append("What maps and tables are included in this document?")

### Challenge 5

In [14]:
from challenge_5 import pdf_optimized_rag_chatbot as prc
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [16]:
chat = prc.PDFChatbot()
chat.load_pdf(pdf_url, collection_name="world_war_II")

Loading PDF from: https://www.pollitecon.com/html/ebooks/Workbook-4-The-Second-World-War.pdf
Splitting text into chunks with metadata extraction...
Creating vector database with metadata...
Collection exists but is empty, adding documents...
Successfully embedded 560 chunks into existing collection
✅ PDF loaded successfully! Ready to answer questions.


True

In [21]:

for query in queries:
    print(f"📝 Query: {query}")

    docs_retriever_mmr_compress_filter_metadata = chat.retrieve_context(
        query=query,
        search_type="mmr",
        re_rank_compress=True,
        use_metadata=False
    )

    print(f"📊 Retrieved {len(docs_retriever_mmr_compress_filter_metadata)} documents")

    context = [doc.page_content + '\n' for doc in docs_retriever_mmr_compress_filter_metadata]

    prompt = ChatPromptTemplate.from_template(
        "Answer the question based on the provided context."
        "Be concise and accurate.\n\nQuestion: {query}\n\nContext: {context}"
    )
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"query": query, "context": context})

    print("\n💬 Response:")
    # Wrap text for better readability
    import textwrap
    wrapped_response = textwrap.fill(response, width=56)
    for line in wrapped_response.split('\n'):
        print(f"│ {line:<56} │")
    print("=" * 60)

📝 Query: What did Metaxas say when Italy attacked Greece in October 1940?
retriever kwargs: {'fetch_k': 24, 'lambda_mult': 0.7}
📊 Retrieved 4 documents

💬 Response:
│ When Italy attacked Greece in October 1940, Metaxas      │
│ replied "OXI" ("NO") to Italy's ultimatum and later      │
│ addressed Greek newspaper editors, discussing Turkey's   │
│ alliance with England, Bulgaria's cautious stance, and   │
│ his belief that time favored the opponents of the Axis   │
│ powers. He also referenced following the policy of       │
│ Venizelos regarding Greece's alignment with the ocean as │
│ a unifying force.                                        │
📝 Query: Who were the main resistance leaders in Albania during WWII?
retriever kwargs: {'fetch_k': 24, 'lambda_mult': 0.7}
📊 Retrieved 4 documents

💬 Response:
│ The main resistance leader in Albania during WWII was    │
│ Sejfulla Maleshova, a prominent communist leader of the  │
│ Albanian antifascist resistance.                         │
📝 Q

### Graph RAG

In [6]:
for query in queries:
    print(f"\nProcessing Query: {query}")

    result = qa_chain.retrieve_and_answer(query)

    # Process answer with improved formatting
    query = result.get('cypher_query', None)
    answer = json.loads(result.get('answer', None))
    success = result.get('success', None)
    response = answer.get('result', None)
    claim_support_rate = answer.get('claim_support_rate', None)
    contradiction_rate = answer.get('contradiction_rate', None)
    numeric_grounding_rate = answer.get('numeric_grounding_rate', None)

    # Apply decision gate
    decision = qa_chain.decision_gate(claim_support_rate, contradiction_rate, numeric_grounding_rate)

    print("=" * 60)
    print("🔍 RAG QUERY RESULTS")
    print("=" * 60)

    if decision == "Answer":
        print(f"✅ Status: {decision}")
        print("\n💬 Response:")
        print("-" * 20)
        print(f"{response}")
    else:
        print(f"⚠️  Status: {decision}")
        print(f"🔍 Generated Query:")
        print("-" * 20)
        print(f"{query}")
        print(f"\n📊 Decision Reason: {decision}")

    print("=" * 60)


Processing Query: What did Metaxas say when Italy attacked Greece in October 1940?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a)-[r]-(b)
WHERE LOWER(a.name) CONTAINS LOWER("metaxas") OR LOWER(b.name) CONTAINS LOWER("metaxas") OR LOWER(a.context) CONTAINS LOWER("italy") OR LOWER(b.context) CONTAINS LOWER("greece") OR LOWER(a.context) CONTAINS LOWER("october 1940") OR LOWER(b.context) CONTAINS LOWER("october 1940")
RETURN a.name, a.source_context, type(r), b.name, b.source_context, a.chunk_id, b.chunk_id
LIMIT 50

Full Context:
[{'a.name': 'Metaxas', 'a.source_context': "5Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 13\nChronology . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17\nMap 1: World War II, 1939-1942 . . . . . . . . . . . . . . . . . .

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: FOUGHT)} {position: line: 2, column: 22, offset: 28} for query: 'cypher\nMATCH (a)-[r:OPPOSED|FOUGHT]-(b)\nWHERE LOWER(a.name) CONTAINS LOWER("german") OR LOWER(b.name) CONTAINS LOWER("german") OR LOWER(a.context) CONTAINS LOWER("greece") OR LOWER(b.context) CONTAINS LOWER("forces")\nRETURN a.name, a.source_context, type(r), b.name, b.source_context, a.chunk_id, b.chunk_id\nLIMIT 40\n'


Generated Cypher:
cypher
MATCH (a)-[r:OPPOSED|FOUGHT]-(b)
WHERE LOWER(a.name) CONTAINS LOWER("german") OR LOWER(b.name) CONTAINS LOWER("german") OR LOWER(a.context) CONTAINS LOWER("greece") OR LOWER(b.context) CONTAINS LOWER("forces")
RETURN a.name, a.source_context, type(r), b.name, b.source_context, a.chunk_id, b.chunk_id
LIMIT 40

Full Context:
[{'a.name': 'Albanian Partisans', 'a.source_context': "III-1. Military law practiced by the Albanian Partisans . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 85\nv32. Partisan 4th division in the mountains of Montenegro, 1944 . . . . . . . . . . . . . . . . . . . . . . . . . . . . 86\n III-2. The treatment of the prisoners by the Albanian Nationalists . . . . . . . . . . . . . . . . . . . . . . . . . . . . 86\nv33. Romanian soldiers on camels in the Caucaz battles . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 86\nIIIb. Brutalities . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

### Multimodal RAG

In [11]:
# Process multimodal RAG queries with improved formatting
print("🖼️  MULTIMODAL RAG PROCESSING")
print("=" * 60)

for i, query in enumerate(queries, 1):
    print(f"\n🔍 Query #{i}:")
    print(f"   ❓ {query}")
    print("-" * 50)
    
    # Process query and retrieve similar chunks
    embed_query = processor.embed_text(query)
    result = vector_store.similarity_search_by_vector(embed_query, top_k=3)
    
    print(f"📊 Retrieved {len(result)} similar chunks")
    
    # Generate response using multimodal RAG
    response = rag.multimodal_pdf_rag_pipeline(query, result)
    
    print("💬 Response:")
    print("┌" + "─" * 58 + "┐")
    
    # Wrap text for better readability
    import textwrap
    wrapped_response = textwrap.fill(response, width=56)
    for line in wrapped_response.split('\n'):
        print(f"│ {line:<56} │")
    
    print("└" + "─" * 58 + "┘")
    
    if i < len(queries):
        print("\n" + "=" * 60)

print(f"\n✅ Processed {len(queries)} queries successfully!")

🖼️  MULTIMODAL RAG PROCESSING

🔍 Query #1:
   ❓ What did Metaxas say when Italy attacked Greece in October 1940?
--------------------------------------------------
📊 Retrieved 4 similar chunks
💬 Response:
┌──────────────────────────────────────────────────────────┐
│ **Question:** What did Metaxas say when Italy attacked   │
│ Greece in October 1940?  **Answer:**   When Italy        │
│ attacked Greece in October 1940, Prime Minister Ioannis  │
│ Metaxas replied to the Italian ultimatum with a single   │
│ word: **"OXI" ("NO")**. This refusal is commemorated as  │
│ "Oxi Day," Greece’s second national holiday.             │
└──────────────────────────────────────────────────────────┘


🔍 Query #2:
   ❓ Who were the main resistance leaders in Albania during WWII?
--------------------------------------------------
📊 Retrieved 4 similar chunks
💬 Response:
┌──────────────────────────────────────────────────────────┐
│ The main resistance leaders in Albania during WWII were  │
│ **Enver Hox